<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [ ]:
# !pip install pyspark
# !pip install findspark

In [ ]:
# Import libraries

In [1]:
import findspark
import pandas as pd

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

findspark.init()

In [ ]:
# Start session

In [2]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

24/06/12 17:12:15 WARN Utils: Your hostname, Alejandros-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.8 instead (on interface en0)
24/06/12 17:12:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/12 17:12:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/12 17:12:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [4]:
!curl -O "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv" && mv searchterms.csv data/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  227k  100  227k    0     0   157k      0  0:00:01  0:00:01 --:--:--  157k


In [ ]:
# Load the csv into a spark dataframe

In [5]:
searchterms_df = pd.read_csv('data/searchterms.csv')
searchterms = spark.createDataFrame(searchterms_df)

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [6]:
count_rows = searchterms.count()
count_cols = len(searchterms.columns)
print(f'Total number of rows: {count_rows}')
print(f'Total number of columns: {count_cols}')

Total number of rows: 10000
Total number of columns: 4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [7]:
searchterms.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [8]:
searchterms.select("searchterm").printSchema()

root
 |-- searchterm: string (nullable = true)



In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [9]:
searchterms.createOrReplaceTempView("searchterms")
spark.sql(
    """
    SELECT COUNT(*) AS gaming_laptop
    FROM searchterms
    WHERE searchterm = 'gaming laptop'
    """
).show()

+-------------+
|gaming_laptop|
+-------------+
|          499|
+-------------+



In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [10]:
spark.sql(
    """
    SELECT searchterm, COUNT(*) as count
    FROM searchterms
    GROUP BY searchterm
    ORDER BY COUNT(*) DESC
    LIMIT 5
    """
).show()

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [15]:
!curl -O https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz 


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1490  100  1490    0     0   1676      0 --:--:-- --:--:-- --:--:--  1676


In [16]:
!tar --no-same-owner -xvzf model.tar.gz

x sales_prediction.model/
x sales_prediction.model/metadata/
x sales_prediction.model/metadata/part-00000
x sales_prediction.model/metadata/.part-00000.crc
x sales_prediction.model/metadata/_SUCCESS
x sales_prediction.model/data/
x sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet
x sales_prediction.model/data/_SUCCESS
x sales_prediction.model/data/.part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet.crc
x sales_prediction.model/data/._SUCCESS.crc: Failed to restore metadata
tar: Error exit delayed from previous errors.


In [17]:
! mv sales_prediction.model/ data/

In [18]:
! mv model.tar.gz data/

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [21]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegressionModel  # Import LinearRegressionModel

# Load the model
model = LinearRegressionModel.load('data/sales_prediction.model')


In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [22]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    data = [[year, 0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features', 'year')
    predictions = model.transform(__)
    predictions.select('prediction').show()

In [23]:
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



24/06/12 17:31:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
